In [ ]:
# Code ported from laptop onto 10.12.68.72 starting on 8/24/2020 (Gregory Rouze)

# To-do: 
# 1) need to separate user functions and main code - I have done this successfully in the offline version, but I'm having a
# little more trouble in the cloud version
# 2) Add comments on putpose of individual user functions
# 3) Run a long term experiment this weekend to see if I run into VPN problems (processing will take a while)

# v2 differs from v1 in that excess code that I used to prototype this in the cloud (from my local laptop)
# was removed.


In [1]:
'''Import relevant packages, functions, and user functions used in this reference ET derivation'''

import boto3
from contextlib import contextmanager 
import earthpy.spatial as es
import fsspec
from math import e
import rasterio as rio
import xarray as xr
from osgeo.gdalnumeric import *
from osgeo.gdalconst import *
import os
from osgeo import gdal, osr, gdal_array, gdalconst
import pandas as pd
import re
import numpy as np
import sys
import ogr
from rasterio import Affine, MemoryFile
from rasterio.enums import Resampling
import rioxarray
from shapely.geometry import Point, Polygon
import geopandas as gpd
from shapely.geometry import box
from fiona.crs import from_epsg
from matplotlib import pyplot as plt
from rasterio.plot import plotting_extent
import earthpy.plot as ep
import math
from itertools import chain
from ipynb.fs.full.ProjectedReferenceET_Classes_Functions import ET0_PM, aggregate_raster_inmem, resample_raster_write, \
reproject_raster, grepfxn, rastermath, lapply_brick, write_geotiff, atmospheric_pressure, relative_fromspecific, unique, s3_push_delete_local
from ipynb.fs.full.ProjectedReferenceET_Classes_Functions import *
import boto3

'''Set home path if not done so already'''

os.getcwd()
os.chdir('/home/jupyter-rouze')

In [2]:
'''Read configuration file and parse out the inputs line by line'''

# Note that the difference between historical and future outputs in cloud are based on these 2 configuration files.
configurationfile = 'configurationfile_referenceET_test_future.ini'
# configurationfile = 'configurationfile_referenceET_test_historical.ini'

# Note: if you want run rcp 8.5, then all you have to do is change the rcp_source parameter from within config file
# It only affects grepfxn(rcp_source,all_files) below

with open(configurationfile) as f:
    data = {}
    for line in f:
        key, value = line.strip().split(' = ')
        data[key] = value
        
print(data)

model_files = data['model_files']
data_source = data['data_source']
output_folder = data['output_folder']
elevfile = data['elevfile']
tiffolder = data['tiffolder']
ET0_method = data['ET0_method']
ET0_winddat = data['ET0_winddat']
ET0_crop = data['ET0_crop']
to_clip = data['to_clip']
model = data['model']
northmost = float(data['northmost'])
southmost = float(data['southmost'])
westmost = float(data['westmost'])
eastmost = float(data['eastmost'])
pad_factor = float(data['pad_factor'])
rcp_source = data['rcp_source']
MACA_start_bucket = data['MACA_start_bucket']

'''This is needed to retrieve the netCDF files from the dev-et-data AWS bucket'''
# os.chdir(model_files)
fs = fsspec.filesystem(model_files, anon=False, requester_pays=True)

all_files = fs.find(MACA_start_bucket)

# This prints all of the files in dev-et-data/in/DelawareRiverBasin/ or MACA_start_bucket...a big set of outputs, so skipped
# print(all_files)

{'model_files': 's3', 'data_source': 'METDATA', 'output_folder': 'in', 'elevfile': 'in/Elevation/cgiar_srtmmerge.tif', 'tiffolder': 'C:/Users/GRouze/Desktop/ReferenceET_Python/Future-MACA/Individual/geotiffs', 'ET0_method': 'PM', 'ET0_winddat': 'yes', 'ET0_crop': 'short', 'to_clip': 'True', 'model': 'MIROC5', 'northmost': '42.54', 'southmost': '38.6', 'westmost': '-76.3', 'eastmost': '-74.23', 'pad_factor': '0.50', 'rcp_source': 'rcp45', 'bucket_filepath_root': 'dev-et-data/out/DelawareRiverBasin/', 'bucket_filepath_branch': 'Run08262020_Firstattemptcloudpush', 'MACA_start_bucket': 'dev-et-data/in/DelawareRiverBasin/'}


In [ ]:
# THE CODE BELOW IS PARSED FROM THE CONDIITION WHEN DEALING WITH METDATA

In [3]:
# Split models apart that are to be used for ensemble averaging
models_parsed = [x.strip() for x in model.split(',')]

# Whittle down the number of files if the folder contains both rcp 4.5 and rcp 8.5 files
# Right now, the code can only handle one model of METDATA output (8/21/2020)
rcp_all_files = [grepfxn(rcp_source,all_files)][0]

# Iterate the files by each each specified model
models_list=[]
for i in range(len(models_parsed)):
    model_files_loop = [grepfxn(models_parsed[i],rcp_all_files)][0]
    models_list.append(model_files_loop)
        
# Flatten series of lists into one list
rcp_all_files = list(chain(*models_list))

# prints all netCDF files from 1950-2100 from MACA (radiation, precipitation, wind etc.)
print(rcp_all_files)


['dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2011_2015_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2016_2020_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2021_2025_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2026_2030_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2031_2035_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2036_2040_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2041_2045_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1

In [4]:
# Find and compile the year blocks into a list
dfis=[]
for out in rcp_all_files:
    a=out.split('_')
    dfi = a[5]+'_'+a[6]
    dfis.append(dfi)

# print(dfis)
    
# Distill the above list into unique year blocks, as there will be duplicates from multiple climate inputs
year_all=unique(dfis);print(year_all)

# For prototyping only
year_block=0
# print(year_all)
# Print the first entry in the year list
print(year_all[year_block])

['2006_2010', '2011_2015', '2016_2020', '2021_2025', '2026_2030', '2031_2035', '2036_2040', '2041_2045', '2046_2050', '2051_2055', '2056_2060', '2061_2065', '2066_2070', '2071_2075', '2076_2080', '2081_2085', '2086_2090', '2091_2095', '2096_2099']
2006_2010


In [ ]:
# Take out the components of the for loop below for showcasing to other members of the ET group 

# loop by each block associated with the MACA netCDF file naming structure
for year_block in range(0,len(year_all)):

    year_block_files = grepfxn(year_all[year_block],rcp_all_files)
    
    print(year_block_files)

    bounds=[southmost,northmost,westmost,eastmost]

    rcp_pr = lapply_brick(grepfxn("pr",year_block_files), 'precipitation', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # downwelling shortwave radiation
    rcp_rsds = lapply_brick(grepfxn("rsds",year_block_files), 'surface_downwelling_shortwave_flux_in_air', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # maximum air temperature
    rcp_tasmax = lapply_brick(grepfxn("tasmax",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # minimum air temperature
    rcp_tasmin = lapply_brick(grepfxn("tasmin",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
    # Now repeat above for the rcp 8.5 model outputs below

    if(data_source == 'METDATA'):

        rcp_uas = lapply_brick(grepfxn("uas",year_block_files), 'eastward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

        rcp_vas = lapply_brick(grepfxn("vas",year_block_files), 'northward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

        rcp_rhsmax = lapply_brick(grepfxn("rhsmax",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

        rcp_rhsmin = lapply_brick(grepfxn("rhsmin",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

        # The section below is meant to convert netCDF files into geoTIFFs
        
    src = rio.open(elevfile)
    # elevation_full_aggregate = aggregate_raster_inmem(src,scale=0.5)
    aggoutput_name='elevation_aggregated.tif'
    
    resample_raster_write(src, name= aggoutput_name,scale=0.5)
    
    dst_filename='elevation_aggregated_resampled.tif'
    match_filename=rcp_pr[0][0].name
    reproject_raster(aggoutput_name, match_filename,dst_filename)
    
    elevation_array=rio.open(dst_filename).read(1)

    # from datetime import datetime - will need to update to make start/end adaptive (7/28/2020)
#     start_year=year_all[year_block][0:4]
#     end_year=year_all[year_block][5:9]
    start_year=year_all[year_block][0:4]
    end_year=year_all[year_block][5:9]

    start=start_year+'-01-01'
    end=end_year+'-12-31'
    datetimes = pd.date_range(start=start,end=end)
#     i=10
    
    for i in range(0,rcp_pr[0][0].count):

        doy_loop = pd.Period(datetimes[i],freq='D').dayofyear
        year_loop = pd.Period(datetimes[i],freq='D').year

        # step 1: extract ith band from the raster stack
        # step 2: stack those ith bands together
        # step 3: do raster mean math from step 2
        pr_stack=[]

        # Purpose: create stacks of variables individually - this is like brick in R
        pr_ensemble = []
        rsds_ensemble = []
        tasmax_ensemble = []
        tasmin_ensemble = []

        j = 0

        # should be 1 array for each variable (mean of x ensembles for a given doy)
        # rcp_pr[0][0].read(1, masked=False).shape
        rcp_pr_doy = rastermath(rcp_pr[0], i)
        rcp_rsds_doy = rastermath(rcp_rsds[0], i)
        rcp_tasmax_doy = rastermath(rcp_tasmax[0], i)
        rcp_tasmin_doy = rastermath(rcp_tasmin[0], i)

        dims = np.shape(rcp_pr_doy[0])
        rows = dims[0]
        cols = dims[1]
        constant_1_dat = np.full((rows,cols), 17.27)
        constant_2_dat = np.full((rows,cols), 0.6108)
        constant_3_dat = np.full((rows,cols), 273.15)
        constant_4_dat = np.full((rows,cols), 237.3)

        rcp_vs_tmax_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmax_doy[0]-constant_3_dat) / ( (rcp_tasmax_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
        rcp_vs_tmin_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmin_doy[0]-constant_3_dat) / ( (rcp_tasmin_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
        rcp_saturatedvapor_doy = (rcp_vs_tmax_array + rcp_vs_tmin_array)/2 

        if(data_source == 'METDATA'): # line 180 from R script

            # All of these are arrays by the way
            rcp_rhsmax_doy = rastermath(rcp_rhsmax[0], i)
            rcp_rhsmin_doy = rastermath(rcp_rhsmin[0], i)
            rcp_uas_doy = rastermath(rcp_uas[0], i)
            rcp_vas_doy = rastermath(rcp_vas[0], i)

            # was below are just arrays, not metadata profiles
            rcp_was_doy_10m = np.sqrt(rcp_uas_doy[0]**2 + rcp_vas_doy[0]**2 )

            rcp_actualvapor_doy = (rcp_vs_tmin_array * rcp_rhsmax_doy[0]/100 + rcp_vs_tmax_array * rcp_rhsmin_doy[0]/100)/2

        da = xr.open_rasterio(rcp_pr[1])
        da_r = rio.open(rcp_pr[1])
        ny, nx = len(da['y']), len(da['x'])
        longitude_array, latitude_array = np.meshgrid(da['x'], da['y'])

        latitude_array_rad = latitude_array * (math.pi/180)

        # Wind speed at 2 meters
        z = np.full((rows,cols), 10)
        array_487 = np.full((rows,cols), 4.87)
        array_678 = np.full((rows,cols), 67.8)
        array_542 = np.full((rows,cols), 5.42)

        if (data_source == 'METDATA'):
            rcp_was_doy_2m = rcp_was_doy_10m * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman
        else:
            rcp_was_doy_2m = rcp_was_doy_10m[0] * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman

        doy_array = np.full((rows,cols), i+1)

        rcp_pr_doy[1]['count']=1
        rcp_tasmin_doy[1]['count']=1
        rcp_tasmax_doy[1]['count']=1

        # To-do: go ahead and developed ET0 directly as opposed to the R implementation(7/29)

        ET0_inputarrays_rcp = [rcp_pr_doy[0], rcp_rsds_doy[0], rcp_tasmin_doy[0],
                                     rcp_tasmax_doy[0],rcp_was_doy_2m,rcp_saturatedvapor_doy,
                                     rcp_actualvapor_doy,elevation_array,latitude_array_rad,doy_array]

        # NameError: name 'ET0_method' is not defined
        if ET0_method  == "yes":
          if ET0_crop != "short" and ET0_crop != "tall":
            stop("Please enter 'short' or 'tall' for the desired reference crop type")
          else:
            alpha = 0.23 # albedo for both short and tall crop
            if (ET0_crop == "short"):
              z0 = 0.02 # roughness height for short grass
            else:
              z0 = 0.1 # roughness height for tall grass
        else:
            z0 = 0.02 # roughness height for short grass
            alpha = 0.25 # semi-desert short grass - will not be used for calculation - just informative

        constants=[alpha, z0]

        ET0_rcp = ET0_PM(ET0_inputarrays_rcp,ET0_method,ET0_winddat,ET0_crop,constants)
        ET0_rcp.incoming_shortwave()
        ET0_rcp.outgoing_shortwave()
        ET0_rcp.outgoing_longwave()
        ET0_rcp.net_radiation()
        ET0_rcp_array_from_class = ET0_rcp.ET0_calcs()
        ET0_rcp_array_final = ET0_rcp_array_from_class.astype('float32')

        rcp_pr_doy[1]['count']=1

        os.chdir('/home/jupyter-rouze')
        gTIFF_filename = write_geotiff(data=ET0_rcp_array_final,meta=rcp_pr_doy[1],var_name='reference_evapotranspiration',
                                       doy=doy_loop,year=year_loop,folder=output_folder)

        local_file = output_folder+'/' + 'reference_evapotranspiration' + '/' + gTIFF_filename 
        bucket = 'dev-et-data'

        bucket_filepath = 'in/DelawareRiverBasin/ETo/'+ str(year_loop)  + '/' + gTIFF_filename 

        os.chdir('/home/jupyter-rouze')
        s3_push_delete_local(local_file, bucket, bucket_filepath)


In [ ]:
##################### Break down for loop by components for showcasing

In [5]:
# range(0,len(year_all))
year_block_files = grepfxn(year_all[0],rcp_all_files)
print(year_block_files)

bounds=[southmost,northmost,westmost,eastmost]

'''For a given input netCDF file, lapplybrick() creates a rasterio object that is clipped specifically for Delaware'''

rcp_pr = lapply_brick(grepfxn("pr",year_block_files), 'precipitation', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# downwelling shortwave radiation
rcp_rsds = lapply_brick(grepfxn("rsds",year_block_files), 'surface_downwelling_shortwave_flux_in_air', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# maximum air temperature
rcp_tasmax = lapply_brick(grepfxn("tasmax",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# minimum air temperature
rcp_tasmin = lapply_brick(grepfxn("tasmin",year_block_files), 'air_temperature', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)
# Now repeat above for the rcp 8.5 model outputs below

if(data_source == 'METDATA'):

    rcp_uas = lapply_brick(grepfxn("uas",year_block_files), 'eastward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

    rcp_vas = lapply_brick(grepfxn("vas",year_block_files), 'northward_wind', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

    rcp_rhsmax = lapply_brick(grepfxn("rhsmax",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

    rcp_rhsmin = lapply_brick(grepfxn("rhsmin",year_block_files), 'relative_humidity', model_files,tiffolder,data_source,to_clip=to_clip,bounds=bounds,pad_factor=pad_factor)

    # The section below is meant to convert netCDF files into geoTIFFs

print(rcp_pr)

['dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/RAD/Climatology/macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/RH/RHSMAX/Climatology/macav2metdata_rhsmax_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/RH/RHSMIN/Climatology/macav2metdata_rhsmin_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/Temp/Climatology/Tmax/macav2metdata_tasmax_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/Temp/Climatology/Tmin/macav2metdata_tasmin_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/WIND/UAS/Climatology/macav2metdata_uas_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/WIND/VAS/Climatology/macav2metdata_vas_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc']
([<open DatasetReader nam

In [6]:
'''Read and resample elevation to match raster characteristics from all other MACA inputs needed for reference ET'''

src = rio.open(elevfile)
# elevation_full_aggregate = aggregate_raster_inmem(src,scale=0.5)
aggoutput_name='elevation_aggregated.tif'

resample_raster_write(src, name= aggoutput_name,scale=0.5)

dst_filename='elevation_aggregated_resampled.tif'
match_filename=rcp_pr[0][0].name
reproject_raster(aggoutput_name, match_filename,dst_filename)

elevation_array=rio.open(dst_filename).read(1)

# from datetime import datetime - will need to update to make start/end adaptive (7/28/2020)
start_year=year_all[year_block][0:4]
end_year=year_all[year_block][5:9]

start=start_year+'-01-01'
end=end_year+'-12-31'
datetimes = pd.date_range(start=start,end=end)
# i=10

print(start)
print(end)

2006-01-01
2010-12-31


In [7]:
# for i in range(0,rcp_pr[0][0].count):
    
doy_loop = pd.Period(datetimes[0],freq='D').dayofyear
year_loop = pd.Period(datetimes[0],freq='D').year

print(doy_loop)
print(year_loop)

# step 1: extract ith band from the raster stack
# step 2: stack those ith bands together
# step 3: do raster mean math from step 2
pr_stack=[]

# Purpose: create stacks of variables individually - this is like brick in R
pr_ensemble = []
rsds_ensemble = []
tasmax_ensemble = []
tasmin_ensemble = []


1
2006


In [13]:
# Here we are using index = 0 or January 1 of the first year in the given netCDF file (e.g. 2021, 2026 2031, 2036 etc.)

# should be 1 array for each variable (mean of x ensembles for a given doy)
# rcp_pr[0][0].read(1, masked=False).shape

print(rcp_pr)
print(rcp_pr[0]) # rasterio opened geoTIFF of precipitation 
print(rcp_pr[0][0])
'''rastermath() averages across all models...however, since we are only dealing with one model for now (i.e. MIROC5),
the rastermath() APPEARS to be redundant. However, future iterations of this code, such as rastermath(), are expecting
an ensemble average of inputs, which are better for to gauge model uncertainty.'''
rcp_pr_doy = rastermath(rcp_pr[0], i)

# print(rcp_pr_doy)
# print(rcp_pr_doy[0]) # array
# print(rcp_pr_doy[1]) # geoTIFF metadata

([<open DatasetReader name='in/temp/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.tif' mode='r'>], 'in/temp/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.tif')
[<open DatasetReader name='in/temp/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.tif' mode='r'>]
<open DatasetReader name='in/temp/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.tif' mode='r'>


In [ ]:
'''Repeat rastermath() for all other inputs needed for reference ET'''

rcp_rsds_doy = rastermath(rcp_rsds[0], i)
rcp_tasmax_doy = rastermath(rcp_tasmax[0], i)
rcp_tasmin_doy = rastermath(rcp_tasmin[0], i)

dims = np.shape(rcp_pr_doy[0])
rows = dims[0]
cols = dims[1]

'''Derive saturated vapor pressure for Penman-Monteith Approximation'''

constant_1_dat = np.full((rows,cols), 17.27)
constant_2_dat = np.full((rows,cols), 0.6108)
constant_3_dat = np.full((rows,cols), 273.15)
constant_4_dat = np.full((rows,cols), 237.3)

rcp_vs_tmax_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmax_doy[0]-constant_3_dat) / ( (rcp_tasmax_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
rcp_vs_tmin_array = constant_2_dat * np.exp(constant_1_dat * (rcp_tasmin_doy[0]-constant_3_dat) / ( (rcp_tasmin_doy[0]-constant_3_dat) + constant_4_dat)) # Equation S2.5
rcp_saturatedvapor_doy = (rcp_vs_tmax_array + rcp_vs_tmin_array)/2 # s2.6


In [ ]:
if(data_source == 'METDATA'): # line 180 from R script

    # All of these are arrays by the way
    rcp_rhsmax_doy = rastermath(rcp_rhsmax[0], i)
    rcp_rhsmin_doy = rastermath(rcp_rhsmin[0], i)
    rcp_uas_doy = rastermath(rcp_uas[0], i)
    rcp_vas_doy = rastermath(rcp_vas[0], i)

    # was below are just arrays, not metadata profiles
    rcp_was_doy_10m = np.sqrt(rcp_uas_doy[0]**2 + rcp_vas_doy[0]**2 )

    # inputs: min/max saturated vapor pressure from air temp., min/max relative humidity (assuming relative humidity is present)
    rcp_actualvapor_doy = (rcp_vs_tmin_array * rcp_rhsmax_doy[0]/100 + rcp_vs_tmax_array * rcp_rhsmin_doy[0]/100)/2 # s2.7


In [ ]:
da = xr.open_rasterio(rcp_pr[1])
da_r = rio.open(rcp_pr[1])
ny, nx = len(da['y']), len(da['x'])
longitude_array, latitude_array = np.meshgrid(da['x'], da['y'])

# Latitude (needed for Extraterrestrial radiation or Ra), in radians
latitude_array_rad = latitude_array * (math.pi/180)

# Convert from wind speed at 10 meters to wind speed at 2 meters
z = np.full((rows,cols), 10)
array_487 = np.full((rows,cols), 4.87)
array_678 = np.full((rows,cols), 67.8)
array_542 = np.full((rows,cols), 5.42)

if (data_source == 'METDATA'):
    rcp_was_doy_2m = rcp_was_doy_10m * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman
else:
    rcp_was_doy_2m = rcp_was_doy_10m[0] * array_487 / np.log(array_678*z - array_542) # Equation S5.20 for PET formulations other than Penman

doy_array = np.full((rows,cols), i+1)

rcp_pr_doy[1]['count']=1
rcp_tasmin_doy[1]['count']=1
rcp_tasmax_doy[1]['count']=1

# To-do: go ahead and developed ET0 directly as opposed to the R implementation(7/29)

In [ ]:
# To-do: go ahead and developed ET0 directly as opposed to the R implementation(7/29)

# Combine all of the inputs into an list of arrays

ET0_inputarrays_rcp = [rcp_pr_doy[0], rcp_rsds_doy[0], rcp_tasmin_doy[0],
                             rcp_tasmax_doy[0],rcp_was_doy_2m,rcp_saturatedvapor_doy,
                             rcp_actualvapor_doy,elevation_array,latitude_array_rad,doy_array]

# NameError: name 'ET0_method' is not defined
if ET0_method  == "yes":
  if ET0_crop != "short" and ET0_crop != "tall":
    stop("Please enter 'short' or 'tall' for the desired reference crop type")
  else:
    alpha = 0.23 # albedo for both short and tall crop
    if (ET0_crop == "short"):
      z0 = 0.02 # roughness height for short grass
    else:
      z0 = 0.1 # roughness height for tall grass
else:
    z0 = 0.02 # roughness height for short grass
    alpha = 0.25 # semi-desert short grass - will not be used for calculation - just informative

constants=[alpha, z0]

'''Initialize the ET0 class from the imported  Jupyter notebook external to this one (see top for imports)'''
ET0_rcp = ET0_PM(ET0_inputarrays_rcp,ET0_method,ET0_winddat,ET0_crop,constants)
ET0_rcp.incoming_shortwave()
ET0_rcp.outgoing_shortwave()
ET0_rcp.outgoing_longwave()
ET0_rcp.net_radiation()
ET0_rcp_array_from_class = ET0_rcp.ET0_calcs()
ET0_rcp_array_final = ET0_rcp_array_from_class.astype('float32')

In [ ]:
print(ET0_rcp_array_final)

result = ET0_rcp_array_final.ravel() 
cleanedList = [x for x in result if str(x) != 'nan']

from scipy import stats
stats.describe(cleanedList)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(ET0_rcp_array_final)
plt.show()
plt.title("Histogram with 'auto' bins")
plt.hist(ET0_rcp_array_final)
plt.show()

In [ ]:
rcp_pr_doy[1]['count']=1

os.chdir('/home/jupyter-rouze')
gTIFF_filename = write_geotiff(data=ET0_rcp_array_final,meta=rcp_pr_doy[1],var_name='reference_evapotranspiration',
                               doy=doy_loop,year=year_loop,folder=output_folder)

local_file = output_folder+'/' + 'reference_evapotranspiration' + '/' + gTIFF_filename 
bucket = 'dev-et-data'

bucket_filepath = 'in/DelawareRiverBasin/ETo/'+ str(year_loop)  + '/' + gTIFF_filename 

os.chdir('/home/jupyter-rouze')

In [ ]:
reread = rio.open(local_file)
reread.meta
print(round(reread.meta['transform'][0],4),round(reread.meta['transform'][4],4))

In [ ]:
'''Push newly created geoTIFF into specified bucket and its filepath'''

s3_push_delete_local(local_file, bucket, bucket_filepath)